# Instalaciones previas

In [ ]:
#!pip install requests
#!pip install beautifulsoup4
#!pip install lxml
#!pip install unidecode
#import nltk
#nltk.download('book')
#!pip install xlsxwriter
#nltk.download('stopwords')
#!pip install -U kaleido

# Librerias

In [89]:
# Web Scraping
import requests
from bs4 import BeautifulSoup
import urllib.request


# librerias para Procesamiento del Lenguaje Natural
import re
import nltk
import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.book import *

#### libreria para navegación de carpetas
import glob


# Manejo de datos
import pandas as pd
import numpy as np
import math

# librerias para visualizaciones
import matplotlib.pyplot as plt
from matplotlib import style
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio



#lectura de archivos
import os

Link del sitio web del cual se desea hacer Web Scrapping

In [90]:
url = "https://snies.mineducacion.gov.co/portal/ESTADISTICAS/Bases-consolidadas/"
response = requests.get(url,verify= False)
soup = BeautifulSoup(response.content, 'html.parser')

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'snies.mineducacion.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings



Extracción de los titulos de los conjuntos de datos, asi como el link donde se pueden descargar el respectivo conjunto.

In [91]:
titulos= []
links = []
for article in soup.find_all('tr',class_="rowodd"):
  titulo = article.a['title']
  titulo = titulo[5:].lower()
  link = "https://snies.mineducacion.gov.co/1778/" + article.a['href']
  titulos.append(titulo)
  links.append(link)
  if list(filter(lambda x: '2017' in x, titulos))==[ ]:
    for article in soup.find_all('tr',class_="roweven"):
      titulo = article.a['title']
      titulo = titulo[5:].lower()
      link = "https://snies.mineducacion.gov.co/1778/" + article.a['href']
      titulos.append(titulo)
      links.append(link)



Descarga de forma automatica de todos los archivos del sitio web

In [92]:
for i in range(0, len(links)):
  url = links[i]
  ruta_archivo = f'/content/drive/MyDrive/Pba_Datos_Caterine_Sarmiento/Bases/{titulos[i]}.xlsx'
  response = requests.get(url)

  if response.status_code == 200:
    with open(ruta_archivo, 'wb') as f:
      f.write(response.content)
    print(f"Archivo {titulos[i]} guardado en ruta")
  else:
    print("Error al descargar el archivo")

Archivo estudiantes matriculados en primer curso 2019 guardado en ruta
Archivo estudiantes matriculados 2021 guardado en ruta
Archivo estudiantes inscritos 2021 guardado en ruta
Archivo estudiantes  admitidos 2021 guardado en ruta
Archivo administrativos 2021 guardado en ruta
Archivo estudiantes graduados 2020 guardado en ruta
Archivo estudiantes matriculados 2020 guardado en ruta
Archivo estudiantes admitidos 2020 guardado en ruta
Archivo administrativos 2020 guardado en ruta
Archivo estudiantes matriculados 2019 guardado en ruta
Archivo estudiantes graduados 2019 guardado en ruta
Archivo docentes 2019 guardado en ruta
Archivo estudiantes graduados 2001 a 2017 guardado en ruta
Archivo administrativos 2015 guardado en ruta
Archivo administrativos 2017 guardado en ruta
Archivo docentes hasta 2013 guardado en ruta
Archivo docentes 2015 guardado en ruta
Archivo docentes 2017 guardado en ruta
Archivo estudiantes matriculados hasta 2013 guardado en ruta
Archivo estudiantes matriculados 2015

## Preprocesamiento


### Estudiantes Inscritos

In [93]:
#### path específica la ruta donde estan los archivos que se van a unificar
path = r'/content/drive/MyDrive/Pba_Datos_Caterine_Sarmiento/Bases'

#### glob.glob extrae los nombres de los archivos que se encuentran en la ruta especificada y que son tipo xlsx
filenames = glob.glob(path + "/*.xlsx")

#### filtro de los archivos que contengan la palabra inscritos en su nombre

filenames = list(filter(lambda x: 'inscritos' in x, filenames))
filenames = [elem for elem in filenames if elem.find('13') == -1]
inscritos = pd.DataFrame()

## ciclo que lee cada archivo, realiza el preprocesamiento de las columnas y posteriormente unifica en un único archivo toda la información,
## preprocesada
for file in filenames:
   df =  pd.read_excel(file)
   df = df.dropna(0)
   indice = df.index[0]
   df = pd.read_excel(file,skiprows=indice+1)
   df.columns = df.columns.str.lower()
   df.columns = df.columns.str.replace("á","a")
   df.columns = df.columns.str.replace("é","e")
   df.columns = df.columns.str.replace("í","i")
   df.columns = df.columns.str.replace("ó","o")
   df.columns = df.columns.str.replace("ú","u")
   df.columns = df.columns.str.replace("\n", "")
   df.columns = df.columns.str.replace("_"," ")
   df.columns = df.columns.str.replace("genero","sexo")
   df.columns = df.columns.str.replace("oseccional","seccional")
   df.columns = df.columns.str.replace("dedomicilio","domicilio")
   df.columns = df.columns.str.replace("delprograma","programa")
   df.columns = df.columns.str.replace("delprograma","programa")
   df.rename(columns={'ies        sector ies': 'sector ies'},inplace = True)
   df.rename(columns={'id sector': 'id sector ies'},inplace = True)
   df.rename(columns={'id nivel': 'id nivel academico'},inplace = True)
   df.rename(columns={'ies sector ies': 'sector ies'},inplace = True) 
   df.rename(columns={'codigo del municipio': 'codigo municipio (ies)'},inplace = True)
   df.rename(columns={'ies\tsector ies': 'sector ies'},inplace = True)
   #df.rename(columns={df.columns[df.columns.str.contains("inscri")][0]: f'inscritos{file[-9:-5]}'},inplace = True)
   df.rename(columns={df.columns[df.columns.str.contains("inscri")][0]: 'inscritos'},inplace = True)
   lista = [[palabra for palabra in frase.split() if palabra.lower() not in stopwords.words('spanish')] for frase in df.columns]
   df.columns = [TreebankWordDetokenizer().detokenize(tokens) for tokens in lista]
   df.columns = df.columns.str.replace(" ","")
   df.drop_duplicates()
   inscritos = inscritos.append(
      df)
inscritos


<ipython-input-93-cebb81ff93d9>:17: FutureWarning:

In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.

<ipython-input-93-cebb81ff93d9>:45: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

Exception ignored in: <function ZipFile.__del__ at 0x7f0b042aadd0>
Traceback (most recent call last):
  File "/usr/lib/python3.10/zipfile.py", line 1821, in __del__
    self.close()
  File "/usr/lib/python3.10/zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
<ipython-input-93-cebb81ff93d9>:17: FutureWarning:

In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.

<ipython-input-93-cebb81ff93d9>:45: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-93-cebb81ff93d9>:17: FutureWarning:

In a future version 

,codigoinstitucion,iespadre,institucioneducacionsuperior(ies),principalseccional,idsectories,sectories,idcaracter,caracteries,codigodepartamento(ies),departamentodomicilioies,...,desccinecodigodetallado,codigodepartamento(programa),departamentoofertaprograma,codigomunicipio(programa),municipioofertaprograma,idsexo,sexo,año,semestre,inscritos
0,1101,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,Principal,1,OFICIAL,4,Universidad,11,"Bogotá, D.C.",...,Producción agrícola y ganadera,11,"Bogotá, D.C.",11001,"Bogotá, D.C.",1,Hombre,2021,1,57
1,1101,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,Principal,1,OFICIAL,4,Universidad,11,"Bogotá, D.C.",...,Producción agrícola y ganadera,11,"Bogotá, D.C.",11001,"Bogotá, D.C.",1,Hombre,2021,2,46
2,1101,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,Principal,1,OFICIAL,4,Universidad,11,"Bogotá, D.C.",...,Producción agrícola y ganadera,11,"Bogotá, D.C.",11001,"Bogotá, D.C.",2,Mujer,2021,1,40
3,1101,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,Principal,1,OFICIAL,4,Universidad,11,"Bogotá, D.C.",...,Producción agrícola y ganadera,11,"Bogotá, D.C.",11001,"Bogotá, D.C.",2,Mujer,2021,2,53
4,1101,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,Principal,1,OFICIAL,4,Universidad,11,"Bogotá, D.C.",...,"Fisioterapia, Fonoaudiología, Terapia ocupacio...",11,"Bogotá, D.C.",11001,"Bogotá, D.C.",1,Hombre,2021,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44751,9922,9922,FUNDACION UNIVERSITARIA COMFAMILIAR RISARALDA,Principal,2,PRIVADA,3,Institución Universitaria/Escuela Tecnológica,66,RISARALDA,...,NaN,66,RISARALDA,66001,PEREIRA,1,Hombre,2018,2,10
44752,9922,9922,FUNDACION UNIVERSITARIA COMFAMILIAR RISARALDA,Principal,2,PRIVADA,3,Institución Universitaria/Escuela Tecnológica,66,RISARALDA,...,NaN,66,RISARALDA,66001,PEREIRA,2,Mujer,2018,1,49
44753,9922,9922,FUNDACION UNIVERSITARIA COMFAMILIAR RISARALDA,Principal,2,PRIVADA,3,Institución Universitaria/Escuela Tecnológica,66,RISARALDA,...,NaN,66,RISARALDA,66001,PEREIRA,2,Mujer,2018,2,10
44754,9922,9922,FUNDACION UNIVERSITARIA COMFAMILIAR RISARALDA,Principal,2,PRIVADA,3,Institución Universitaria/Escuela Tecnológica,66,RISARALDA,...,NaN,66,RISARALDA,66001,PEREIRA,1,Hombre,2018,2,11


In [94]:
inscritos.columns

Index(['codigoinstitucion', 'iespadre', 'institucioneducacionsuperior(ies)',
       'principalseccional', 'idsectories', 'sectories', 'idcaracter',
       'caracteries', 'codigodepartamento(ies)', 'departamentodomicilioies',
       'codigomunicipio(ies)', 'municipiodomicilioies', 'codigosniesprograma',
       'programaacademico', 'idnivelacademico', 'nivelacademico',
       'idnivelformacion', 'nivelformacion', 'idmetodologia', 'metodologia',
       'idarea', 'areaconocimiento', 'idnucleo',
       'nucleobasicoconocimiento(nbc)', 'idcinecampoamplio',
       'desccinecampoamplio', 'idcinecampoespecifico',
       'desccinecampoespecifico', 'idcinecodigodetallado',
       'desccinecodigodetallado', 'codigodepartamento(programa)',
       'departamentoofertaprograma', 'codigomunicipio(programa)',
       'municipioofertaprograma', 'idsexo', 'sexo', 'año', 'semestre',
       'inscritos'],
      dtype='object')

### Preprocesamiento


In [95]:
def limpieza(df):
    text_columns = df.select_dtypes(include=['object']).columns
    df[text_columns] = df[text_columns].applymap(lambda x: x.lower() if isinstance(x, str) else x)
    df[text_columns] = df[text_columns].applymap(lambda x: x.replace("á","a") if isinstance(x, str) else x)
    df[text_columns] = df[text_columns].applymap(lambda x: x.replace("é","e") if isinstance(x, str) else x)
    df[text_columns] = df[text_columns].applymap(lambda x: x.replace("í","i") if isinstance(x, str) else x)
    df[text_columns] = df[text_columns].applymap(lambda x: x.replace("ó","o") if isinstance(x, str) else x)
    df[text_columns] = df[text_columns].applymap(lambda x: x.replace("ú","u") if isinstance(x, str) else x)
    df[text_columns] = df[text_columns].applymap(lambda x: x.replace("0","") if isinstance(x, str) else x)
    df[text_columns] = df[text_columns].applymap(lambda x: x.replace("mujer","femenino") if isinstance(x, str) else x)
    df[text_columns] = df[text_columns].applymap(lambda x: x.replace("hombre","masculino") if isinstance(x, str) else x)
    df[text_columns] = df[text_columns].applymap(lambda x: x.replace("no informa","na") if isinstance(x, str) else x)
    df[text_columns] = df[text_columns].applymap(lambda x: x.replace("sin informacion","na") if isinstance(x, str) else x)
    df[text_columns] = df[text_columns].applymap(lambda x: re.sub(r'[^\w\s]', '',x) if isinstance(x, str) else x)
    df["sexo"] = np.where(df["idsexo"]==1,"masculino",np.where(df["idsexo"]==2,"femenino","na"))
    df["nivelformacion"] = np.where(df["nivelformacion"].str.contains("universitaria|universitario"),"universitario",
                                np.where(df["nivelformacion"].str.contains("especializacion"),"especializacion",
                                         np.where(df["nivelformacion"].str.contains("tecnologico|tecnologica"),"tecnologica",
                                                  np.where(df["nivelformacion"].str.contains("no aplica|sin programa especifico"),"sin programa especifico",df["nivelformacion"]))))
    df["metodologia"] = np.where(df["metodologia"].str.contains("presencialvirtual|dual"),"hibrido",
                                np.where(df["metodologia"].str.contains("presencial"),"presencial",
                                         np.where(df["metodologia"].str.contains("virtual"),"virtual",
                                                  np.where(df["metodologia"].str.contains("tradicional|tradicion|tradicio"),"distancia tradicional",
                                                  np.where(df["metodologia"].str.contains("no aplica|sin programa especifico"),"sin programa especifico",df["nivelformacion"])))))
    
    return df


In [96]:
inscritos = limpieza(inscritos)

# Análisis Exploratorio


1) Historico de inscritos

In [97]:
datos = inscritos.groupby(["año"])["inscritos"].sum().reset_index()
fig = px.line(datos, x="año", y="inscritos", title='Historico de Inscritos')
fig.show()

In [98]:
fig.write_html('/content/drive/MyDrive/Pba_Datos_Caterine_Sarmiento/Ima/historico.html')


2) Distribucción del sexo de los inscritos

In [99]:
datos = inscritos.groupby(["sexo"])["inscritos"].sum().reset_index()
fig = px.pie(datos, values='inscritos', names='sexo', title='Sexo Inscritos')
fig.show()

In [100]:
fig.write_html('/content/drive/MyDrive/Pba_Datos_Caterine_Sarmiento/Ima/sexo.html')

3) Inscritos por tipo de Institución educativa

In [101]:
datos = ((inscritos.groupby(["sectories"])["inscritos"].sum())/(inscritos["inscritos"].sum())).reset_index()
fig = px.bar(datos, x='sectories', y='inscritos', title='Inscritos por tipo de Institución Educativa')
fig.show()

In [102]:
fig.write_html('/content/drive/MyDrive/Pba_Datos_Caterine_Sarmiento/Ima/institucion.html')

4) Inscritos por programa academico

In [103]:
datos = ((inscritos.groupby(["nivelacademico","sexo"])["inscritos"].sum())/(inscritos["inscritos"].sum())).reset_index()
fig = px.bar(datos, x="nivelacademico", y="inscritos", color="sexo", title="Inscritos por tipo programa academico")
fig.show()

In [104]:
fig.write_html('/content/drive/MyDrive/Pba_Datos_Caterine_Sarmiento/Ima/tipo_programa.html')

In [105]:
inscritos.to_excel("/content/drive/MyDrive/Pba_Datos_Caterine_Sarmiento/consolidado.xlsx", sheet_name='inscritos', index=False)